<h1 align="center"> pyLDPC Applications: Images Coding & Decoding </h1>

## update:  02/24/16 - v.0.7.0

<b><font color="red"> Since version 0.7: Coding and decoding functions take tG (Transposed G) instead of G, the coding matrix. Functions that construct it (CodingMatrix and CodingMatrix_systematic) return tG instead of G as well. </font></b> 

<br>
<b><font color="blue"> New functions/sections are indicated in blue </font></b>


This notebook introduces the sub-module Images of pyLDPC: ldpc_images.
It explains what each function does and how to use it. If you're more interested in using the module, you may follow the <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Tutorial-Images.ipynb?flush_cache=true"> User's guide: pyLDPC-Images Tutorial</a>

All the functions defined below can be called py importing the sub-module ldpc_images:

```python
e.g :
from  pyldpc import ldpc_images
ldpc_images.Gray2Bin(....)
```

We'll consider 2 types of images in this study:

- Grayscale images: Images whean read by cv2.imread(), are seen as 2D arrays. Each pixel is a uint8 number (0-255). 
- RGB Images: Images that are seen as 3D arrays. Each pixel is an RGB array of 3 uint8 numbers ( [R,G,B] ). 

<font color="blue"><h2> Outline:</h2></font> 
<font color="blue"><h2> --------------------------------------------------</h2></font> 

**I. Images conversion RGB,Gray <=> Binary.**

**II. Images Coding & Decoding:**
 1. Pixel by pixel <font color="blue">*</font><i>(20 times faster compared v0.6)</i>
 2. Row by row using large sparse matrices <font color="red">*</font><i>(slower than pixel coding but more efficient with low SNRs)</i> 
 
**III. Application**
<font color="blue"><h2> --------------------------------------------------</h2></font> 


We'll consider 2 types of images in this study:

- Grayscale images: Images whean read by cv2.imread(), are seen as 2D arrays. Each pixel is a uint8 number (0-255). 
- RGB Images: Images that are seen as 3D arrays. Each pixel is an RGB array of 3 uint8 numbers ( [R,G,B] ). 

To code and decode images, we'll have to "binarize" them first:

- Graysale images are binarized (using Gray2bin function that we'll present later) into 3D arrays where each uint8 number is written as a 8-bits binary array. Therefore, Coding needs a 8-rows coding matrix. The image obtained is shaped: (width, height, 8).
- RGB images are binarized pixel by pixel (using RGB2bin): each color is transformed to an 8-bits binary array, then the 3 arrays obtained of each pixel are gathered into a binary 24-bits array. The image obtained is shaped (width, height, 8).


In [10]:
import warnings
import numpy as np
import scipy
from IPython.display import display, HTML
import pyldpc 
import cv2 
from time import time

# I - RGB, Gray - Binary conversion
First 2 fundamental functions that transform INT <=> Binary Array

In [3]:
def int2bitarray(N,k):
    """
    Changes array's base from int (base 10) to binary (base 2)
    
    Parameters:
    
    N: int N 
    k: Width of the binary array you would like to change N into. N must not be greater than 2^k - 1. 
    
    >> Examples: int2bitarray(6,3) returns [1, 1, 0]
                 int2bitarray(6,5) returns [0, 0, 1, 1,0]
                 int2bitarray(255,8) returns [1, 1, 1, 1, 1, 1, 1, 1]
                 int2bitarray(255,10) returns [0, 0, 1, 1, 1, 1, 1, 1, 1, 1]


    """

    binary_string = bin(N) 
    length = len(binary_string)
    bitarray = np.zeros(k, 'int')
    for i in range(length-2):
        bitarray[k-i-1] = int(binary_string[length-i-1])

    return bitarray

def bitarray2int(bitarray):
    
    """ Changes array's base from binary (base 2) to int (base 10).
    
    Parameters:
    
    bitarray: Binary Array.

    >> Examples: bitarray2int([1, 1, 0]) returns 6
                 bitarray2int([0, 0, 1, 1,0]) returns 6
                 bitarray2int([1, 1, 1, 1, 1, 1, 1, 1]) returns 255
                 

    
    """
    
    bitstring = "".join([str(i) for i in bitarray])
    
    return int(bitstring,2)

## I-1 - Binarizing grayscale images:
Gray2Bin and Bin2Gray transform 2D-graysclae images to 3D-binary arrays

For a grayscale image:
<img src="Equations/Construction19.png">

<i>Gray2Bin</i> returns:

<img src="Equations/Construction20.png">

<i>Bin2Gray </i> does the reverse operation.
   

In [4]:
def Gray2Bin(img):
    """ Puts a GrayScale Image on a binary form 
    
    Parameters:
    
    img_array: 2-D array of a grayscale image (no 3rd dimension)
    
    returns:
    
    3-D img_array in a binary form, each pixel uint8 is transformed to an 8-bits array
    
    
    >>> Example:  the grayscale (2x2) image [[2, 127],    
                                             [255, 0]]
                                        
    will be conveterted to the (2x2x8) binary image:   [[[0, 0, 0, 0, 0, 0, 1, 0],[0, 1, 1, 1, 1, 1, 1, 1]],
                                                        [[1, 1, 1, 1, 1, 1, 1, 1],[0, 0, 0, 0, 0, 0, 0, 0]]]
                                                       
                                                       
    """
    if not len(img.shape)==2:
        raise ValueError('{} must have 2 dimensions. Make sure it\'s a grayscale image.')
        
    height,width = img.shape
    
    img_bin = np.zeros(shape=(height,width,8),dtype=int)
    
    for i in range(height):
        for j in range(width):
            img_bin[i,j,:] = int2bitarray(img[i,j],8)
            
    return img_bin
    

In [5]:
def Bin2Gray(img_bin):
    
    """ Puts a 8-bits binary Image to uint8
    
    Parameters:
    
    img_array: 3-D array (height, width, 8)
    
    returns:
    
    2-D img_array in grayscale
    
    >>> Example:  the (2x2x8) binary image:   [[[0, 0, 0, 0, 0, 0, 1, 0],[0, 1, 1, 1, 1, 1, 1, 1]],
                                                          [[1, 1, 1, 1, 1, 1, 1, 1],[0, 0, 0, 0, 0, 0, 0, 0]]] 
                                            
                                        
    will be conveterted to the (2x2) uint8 image [[2, 127],
                                                  [255, 0]]
    
    """
    height,width,k = img_bin.shape
    img_grayscale = np.zeros(shape=(height,width),dtype=np.uint8)
    
    for i in range(height):
        for j in range(width):
            img_grayscale[i,j] = bitarray2int(img_bin[i,j,:])
            
    return img_grayscale

## I-2- Binarizing RGB Images:
RGB2Bin and Bin2RGB transform 3D colorful images to 3D-binary arrays.

In [5]:
def RGB2Bin(img):
    """ Puts an RGB Image on a binary form 
    
    Parameters:
    
    img_array: 3-D array of an RGB image ( 3rd dimension = 3)
    
    returns:
    
    3-D img_array in a binary form, each pixel is transformed to an 24-bits binary array.
    
    
    >>> Example:  the grayscale (2x1x3) image [[[2, 127,0]],
                                               [[255, 0,1]]]
                                        
    will be conveterted to the (2x1x24) binary image:   
[[[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]],
 [[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]]
                                                       
                                                       
    """
    height,width,depth = img.shape

    if not depth==3:
        raise ValueError('{}\'s 3rd dimension must be equal to 3 (RGB). Make sure it\'s an RGB image.')
      
    
    img_bin = np.zeros(shape=(height,width,24),dtype=int)
    
    for i in range(height):
        for j in range(width):
            R = int2bitarray(img[i,j,0],8)
            G = int2bitarray(img[i,j,1],8)
            B = int2bitarray(img[i,j,2],8)

            img_bin[i,j,:] = np.concatenate((R,G,B))
            
    return img_bin
    

In [1]:
def Bin2RGB(img_bin):
    
    """ Puts a 24-bits binary Image to 3xuint8 (RGB)
    
    Parameters:
    
    img_array: 3-D array (height, width, 24)
    
    returns:
    
    3-D img_array in RGB (height, width, 3)
    
    >>> Example:  the (2x1x24) binary image:   
    
   [[[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]],
   [[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]]
  
                                        
    will be conveterted to the (2x1x3) RGB image :
    
    [[[2, 127,0]],
    [[255, 0,1]]]

    """
    
    height,width,depth = img_bin.shape
    img_rgb = np.zeros(shape=(height,width,3),dtype=np.uint8)
    
    for i in range(height):
        for j in range(width):
            R = bitarray2int(img_bin[i,j,:8])
            G = bitarray2int(img_bin[i,j,8:16])
            B = bitarray2int(img_bin[i,j,16:])

            img_rgb[i,j] = np.array([R,G,B],dtype=np.uint8)
            
    return img_rgb

# II -  Coding and decoding

Coding and decoding images can be done in two different ways:

- Using small matrices: coding/decoding pixel by pixel.
- Using large matrices: coding/decoding row by row, each row being reshaped as a width sized binary array. 

## II - 1 Small matrices: pixel by pixel
 ImageCoding codes a binary image and adds a AWGN with specified SNR. SNR: Signal-Noise Ratio, SNR = 10log(1/variance) in decibels. G has to be systematic if you want to visualize the noisy before decoding. You don't have to specifiy wether the image is grayscale or rgb but the coding matrix G must correspond to the nature of your image.

<font color="red">Since v7.0, ImageCoding takes tG, transposed coding matrix tG instead of G. </font>
<br>

Since the coding is done pixel by pixel, the pixel's size must correspond to k, number of G's rows:

- Grayscale: each pixel is an uint8 number. Which means k must be equal to 8.
- RGB: each pixel is a 3 uint8 numbers tuple. Which means k must be equal to 24. 

The construction of a parity-check matrix (choice of n,d_v,d_c) of a specific (approximate though) so as to get a specific k instructions can be found in the <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Tutorial-Matrices.ipynb?flush_cache=true"> pyLDPC-Matrices Construction Tutorial</a>

### Pixel by pixel coding:

In [6]:
def ImageCoding(tG,img_bin,snr):
    
    """ 
    CAUTION: SINCE V.0.7 Image coding and decoding functions TAKES TRANSPOSED CODING MATRIX tG.
    
    IF G IS LARGE, USE SCIPY.SPARSE.CSR_MATRIX FORMAT TO SPEED UP CALCULATIONS.
    
    Codes a binary image (Therefore must be a 3D-array). Each pixel (k bits-array, k=8 if grayscale, k=24 if colorful) 
    is considered a k-bits message. If the original binary image is shaped (X,Y,k). The coded image will be shaped (X,Y,n)
    Where n is the length of a codeword. Then a gaussian noise N(0,snr) is added to the codeword.
    
    Remember SNR: Signal-Noise Ratio: SNR = 10log(1/variance) in decibels of the AWGN used in coding.

    Of course, showing an image with n-bits array is impossible, that's why if the optional argument show is set to 1, 
    and if Coding Matrix G is systematic, showing the noisy image can be possible by gathering the k first bits of each 
    n-bits codeword to the left, and the redundant bits to the right. Then the noisy image is changed from bin to uint8. 
    Remember that in this case, ImageCoding returns  a tuple: the (X,Y,n) coded image, and the noisy image (X,Y*(n//k)).
    
    Parameters:

    G: Coding Matrix G - (must be systematic to see what the noisy image looks like.) See CodingMatrix_systematic.
    img_bin: 3D-array of a binary image.
    SNR: Signal-Noise Ratio, SNR = 10log(1/variance) in decibels of the AWGN used in coding.
      
    Returns:
    (default): Tuple: noisy_img, coded_img 
    
    """

    n,k = tG.shape
         
    height,width,depth = img_bin.shape 
    
    if k!=8 and k!= 24:
        raise ValueError('Coding matrix must have 8 xor 24 rows ( grayscale images xor rgb images)')
        
        
    coded_img = np.zeros(shape=(height,width,n))
    
    noisy_img = np.zeros(shape=(height,width,k),dtype=int)
    
    for i in range(height):
        for j in range(width):
            coded_byte_ij = Coding(tG,img_bin[i,j,:],snr)
            coded_img[i,j,:] = coded_byte_ij
            systematic_part_ij = (coded_byte_ij[:k]<0).astype(int)

            noisy_img[i,j,:] = systematic_part_ij       
        
    if k==8:
        noisy_img = Bin2Gray(noisy_img)
    else:
        noisy_img = Bin2RGB(noisy_img)

    return coded_img,noisy_img

### Pixel by pixel decoding 
ImageDecoding requires also Coding Matrix G in ordrer to solve v.G = x where x is the codeword element decoded by the function itself if G is not systematic. Otherwise, the first k bits of the codeword are exactly v.

<font color="red">Since v7.0, ImageDecoding takes tG, transposed coding matrix tG instead of G. </font>
<br>

When v is found for each codeword, the decoded image can be transformed from binary to uin8 format if  and shown. Number of rows of G specifies wether the image is in grayscale or rgb format: if k = 8, the image is Grayscale. If k = 24, the image is colorful.

<br>
<br>

<font color="blue"> ImageDecoding uses a slightly different version of decoding functions (pyldpc.Decoding_BP, pyldpc.Decoding_logBP) where some "static" parameters related to H are passed to the decoding functions in order to speed up computing. These optimization changes are hidden so that the user will not suffer from radical syntax changes in new releases.</font>
    

In [7]:
def ImageDecoding(tG,H,img_coded,snr,max_iter=1,log=1):

    """ 

    CAUTION: SINCE V.0.7 Image coding and decoding functions TAKES TRANSPOSED CODING MATRIX tG. 

    IF G IS LARGE, USE SCIPY.SPARSE.CSR_MATRIX FORMAT (IN H AND G) TO SPEED UP CALCULATIONS.

    Image Decoding Function. Taked the 3-D binary coded image where each element is a codeword n-bits array and decodes 
    every one of them. Needs H to decode. A k bits decoded vector is the first k bits of each codeword, the decoded image can
    be transformed from binary to uin8 format and shown.

    Parameters: 

    tG: Transposed coding matrix tG.
    H: Parity-Check Matrix (Decoding matrix). 
    img_coded: binary coded image returned by the function ImageCoding. Must be shaped (heigth, width, n) where n is a
                the length of a codeword (also the number of H's columns)

    snr: Signal-Noise Ratio: SNR = 10log(1/variance) in decibels of the AWGN used in coding.

    log: (optional, default = True), if True, Full-log version of BP algorithm is used. 
    max_iter: (optional, default =1), number of iterations of decoding. increase if snr is < 5db. 


    """
    n,k = tG.shape
    height, width, depth = img_coded.shape

    img_decoded_bin = np.zeros(shape=(height,width,k),dtype = int)

    if log:
        DecodingFunction = Decoding_logBP_ext
    else:
        DecodingFunction = Decoding_BP_ext

    systematic = 1

    if not (tG[:k,:]==np.identity(k)).all():
        warnings.warn("In LDPC applications, using systematic coding matrix G is highly recommanded to speed up decoding.")
        systematic = 0


    BitsNodes = BitsAndNodes(H)
    for i in range(height):
        for j in range(width):

            decoded_vector = DecodingFunction(H,BitsNodes,img_coded[i,j,:],snr,max_iter)
            if systematic:
                decoded_byte = decoded_vector[:k]
            else: 
                decoded_byte = DecodedMessage(tG,decoded_vector)

            img_decoded_bin[i,j,:] = decoded_byte 

    if k==8:
        img_decoded = Bin2Gray(img_decoded_bin)
    else:
        img_decoded = Bin2RGB(img_decoded_bin)

    return img_decoded


## II - 2 Large matrices: row by row

<font color="blue"> ImageCoding_rowbyrow </font> codes a binary image and adds a AWGN with specified SNR. SNR: Signal-Noise Ratio, SNR = 10log(1/variance) in decibels.

To make the use of <font color="blue"> large matrices </font> practical, tG must be in scipy.sparse.csr_matrix format. 
Check the <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Tutorial-Matrices.ipynb?flush_cache=true"> pyLDPC-Matrices Construction Tutorial</a> for more details about how to construct, save and load large matrices. 

<font color="red"> tG must be systematic </font>because solving tG.tv = tx uses a Gaussian elimination on tG which has a O(n^3) complexity and will therefore make the decoding last an eternity. Using a systematic coding matrix saves us this operation by getting v directly from x, since v = x[:k].

Since the coding is done row by row, the row's size (in binary) must correspond to k. 

For example:
a Grayscale image 100x200 has a row's size of 200x8 = 1600 bits. 
an RGB image 100x200 has a row's size of 200x8x3 = 4800 bits. 


The construction of a parity-check matrix (choice of n,d_v,d_c) of a specific (approximate though) so as to get a specific k instructions can be found again, in the <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Tutorial-Matrices.ipynb?flush_cache=true"> pyLDPC-Matrices Construction Tutorial</a>

Using large matrices takes a lot more time than pixel coding using classic numpy calculations. <font color="blue"> Since v0.7, pyLDPC introduces scipy.sparse.csr_matrix format with which matricial products are faster but also guarantees a backward compatibility with numpy arrays. </font>
### Row by row coding:

In [9]:
def ImageCoding_rowbyrow(tG,img_bin,snr):
    
    """ 
    
    CAUTION: SINCE V.0.7 Image coding and decoding functions TAKE TRANSPOSED CODING MATRIX tG. USE
    SCIPY.SPARSE.CSR_MATRIX FORMAT (IN H AND G) TO SPEED UP CALCULATIONS. K MUST BE EQUAL TO THE NUMBER OF BITS IN ONE ROW
    OF THE BINARY IMAGE. USE A SYSTEMATIC CODING MATRIX WITH CodingMatrix_systematic. THEN USE SCIPY.SPARSE.CSR_MATRIX()
    
    --------
    
    Codes a binary image (Therefore must be a 3D-array). Each row of img_bin is considered a k-bits message. If the image has 
    a shape (X,Y,Z) then the binary image will have the shape (X,k). The coded image will be shaped (X+1,n):
    
    - The last line of the coded image stores Y and Z so as to be able to construct the decoded image again via a reshape. 
    - n is the length of a codeword. Then a gaussian noise N(0,snr) is added to the codeword.
    
    Remember SNR: Signal-Noise Ratio: SNR = 10log(1/variance) in decibels of the AWGN used in coding.
 
    ImageCoding returns  a tuple: the (X+1,n) coded image, and the noisy image (X,Y,Z).
    
    Parameters:

    tG: Transposed Coding Matrix G - must be systematic. See CodingMatrix_systematic.
    img_bin: 3D-array of a binary image.
    SNR: Signal-Noise Ratio, SNR = 10log(1/variance) in decibels of the AWGN used in coding.    
    
    Returns:
    (default): Tuple:  coded_img, noisy_img
    
    """
    n,k = tG.shape
    height,width,depth = img_bin.shape
    
    if not type(tG)==scipy.sparse.csr_matrix:
        warnings.warn("Using scipy.sparse.csr_matrix format is highly recommanded when computing row by row coding and decoding to speed up calculations.")
        
    if not (tG[:k,:]==np.identity(k)).all():
        raise ValueError("G must be Systematic. Solving tG.tv = tx for images has a O(n^3) complexity.")
       
    if width*depth != k:
        raise ValueError("If the image's shape is (X,Y,Z) k must be equal to 8*Y (if Gray ) or 24*Y (if RGB)")
    
    img_bin_reshaped = img_bin.reshape(height,width*depth)
    
    coded_img = np.zeros(shape=(height+1,n))
    coded_img[height,0:2]=width,depth
    
    for i in range(height):
        coded_img[i,:] = Coding(tG,img_bin_reshaped[i,:],snr)
        
    noisy_img = (coded_img[:height,:k]<0).astype(int).reshape(height,width,depth)

    
    if depth==8:
        return coded_img,Bin2Gray(noisy_img)
    if depth==24:
        return coded_img,Bin2RGB(noisy_img)

### Row by row decoding 
<font color="blue"> ImageDecoding_rowbyrow</font> doesn't require transposed coding Matrix tG since G must be systematic (Otherwise an error is raised in ImageCoding_rowbyrow) but it makes sense to keep it as an argument:

- Syntax of decoding functions don't change within the same submodule. 
- If a user gives the wrong G as argument (not systematic for instance), an Error is raised (instead of computing a false, long-lasting useless decoding ...) 
- tG stores k. 

    
 <p>   <font color="blue"> ImageDecoding uses a slightly different version of decoding functions (pyldpc.Decoding_BP, pyldpc.Decoding_logBP) where some "static" parameters related to H are passed to the decoding functions in order to speed up computing. These optimization changes are hidden so that the user will not suffer from radical syntax changes in new releases.</font> </p>

In [11]:
def ImageDecoding_rowbyrow(tG,H,img_coded,snr,max_iter=1,log=1):
    
    """ 
        
    CAUTION: SINCE V.0.7 ImageDecoding TAKES TRANSPOSED CODING MATRIX tG INSTEAD OF G. USE SCIPY.SPARSE.CSR_MATRIX 
    FORMAT (IN H AND G) TO SPEED UP CALCULATIONS. 
    
    --------
    Image Decoding Function. Taked the 3-D binary coded image where each element is a codeword n-bits array and decodes 
    every one of them. Needs H to decode and tG to solve tG.tv = tx where x is the codeword element decoded by the function
    itself. When v is found for each codeword, the decoded image can be transformed from binary to uin8 format and shown.
    
    Parameters: 
    
    tG: Transposed Coding Matrix ( SCIPY.SPARSE.CSR_MATRIX FORMAT RECOMMANDED )
    H: Parity-Check Matrix (Decoding matrix).( SCIPY.SPARSE.CSR_MATRIX FORMAT RECOMMANDED)

    img_coded: binary coded image returned by the function ImageCoding. Must be shaped (heigth, width, n) where n is a
                the length of a codeword (also the number of H's columns)
    
    snr: Signal-Noise Ratio: SNR = 10log(1/variance) in decibels of the AWGN used in coding.
    
    log: (optional, default = True), if True, Full-log version of BP algorithm is used. 
    max_iter: (optional, default =1), number of iterations of decoding. increase if snr is < 5db. 

    
    """
    
    n,k = tG.shape
    width,depth = img_coded[-1,0:2]
    img_coded = img_coded[:-1,:]
    height,N = img_coded.shape
    
    if N!=n:
        raise ValueError('Coded Image must have the same number of columns as H')
    if depth !=8 and depth != 24:
        raise ValueError('type of image not recognized: third dimension of the binary image must be 8 for grayscale, or 24 for RGB images')

    if not (tG[:k,:]==np.identity(k)).all():
        raise ValueError("G must be Systematic. Solving tG.tv = tx for images has a O(n^3) complexity")
    
    if not type(H)==scipy.sparse.csr_matrix:
        warnings.warn("Used H is not a csr object. Using scipy.sparse.csr_matrix format is highly recommanded when computing row by row coding and decoding to speed up calculations.")
      
    img_decoded_bin = np.zeros(shape=(height,k),dtype = int)

    if log:
        DecodingFunction = Decoding_logBP
    else:
        DecodingFunction = Decoding_BP
    
    BitsNodes = BitsAndNodes(H)

    for i in range(height):
        decoded_vector = DecodingFunction(H,BitsNodes,img_coded[i,:],snr,max_iter)
        img_decoded_bin[i,:] = decoded_vector[:k]
            
            
    if depth==8:
        img_decoded = Bin2Gray(img_decoded_bin.reshape(height,width,depth))
    if depth==24:
        img_decoded = Bin2RGB(img_decoded_bin.reshape(height,width,depth))
        
    return img_decoded

### Bonus: Bit Error Rate: 
Function that will give us an idea about how accurate our decoding is by comparing the original and the decoded images bit by bit. It returns the rate of false bits over all bits.

In [10]:
def BER(original_img_bin,decoded_img_bin):
    """ 
    
    Computes Bit-Error-Rate (BER) by comparing 2 binary images.
    The ratio of bit errors over total number of bits is returned.
    
    """
    if not original_img_bin.shape == decoded_img_bin.shape:
        raise ValueError('Original and decoded images\' shapes don\'t match !')
        
    height, width, k = original_img_bin.shape 
    
    
    errors_bits = sum(abs(original_img_bin-decoded_img_bin).reshape(height*width*k))
    total_bits  = np.prod(original_img_bin.shape)
    
    BER = errors_bits/total_bits 
    
    return(BER)


# III- Application :
<a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Tutorial-Images.ipynb?flush_cache=true"> User's guide: pyLDPC-Images Tutorial</a>